In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from einops import rearrange
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

from uni2ts.eval_util.plot import plot_single, plot_next_multi
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule

In [2]:
from collections.abc import Generator
from pathlib import Path
from typing import Any

import datasets
import pandas as pd
from datasets import Features, Sequence, Value

In [3]:
COLS = ["mean_swe", "mean_pr", "mean_tair", "mean_vs", "mean_srad", "mean_hum"]

In [4]:
data_path = "model_ready_huc170103010101.csv"
df = pd.read_csv(data_path, index_col=0, parse_dates=True)
df = df[COLS]
df.columns

Index(['mean_swe', 'mean_pr', 'mean_tair', 'mean_vs', 'mean_srad', 'mean_hum'], dtype='object')

In [5]:
# Load dataframe
url_wide = (
    "https://gist.githubusercontent.com/rsnirwan/c8c8654a98350fadd229b00167174ec4"
    "/raw/a42101c7786d4bc7695228a0f2c8cea41340e18f/ts_wide.csv"
)
df = pd.read_csv(url_wide, index_col=0, parse_dates=True)

df.head()

def example_gen_func() -> Generator[dict[str, Any]]:
    for i in range(len(df.columns)):
        yield {
            "target": df.iloc[:, i].to_numpy(),  # array of shape (time,)
            "start": df.index[0],
            "freq": pd.infer_freq(df.index),
            "item_id": f"item_{i}",
        }


features = Features(
    dict(
        target=Sequence(Value("float32")),
        start=Value("timestamp[s]"),
        freq=Value("string"),
        item_id=Value("string"),
    )
)

hf_dataset = datasets.Dataset.from_generator(example_gen_func, features=features)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
hf_dataset.features.copy()

{'target': Sequence(feature=Value(dtype='float32', id=None), length=-1, id=None),
 'start': Value(dtype='timestamp[s]', id=None),
 'freq': Value(dtype='string', id=None),
 'item_id': Value(dtype='string', id=None)}